In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split


df = pd.read_csv("../data/cleaned_data_HELLOIMGREGOIRE_test.csv")
# Selecting parameters to train our price prediction on
df_selection = df[
    [
        "price",
        "living_area",
        "building_type",
        "terrace",
        "garden",
        "swimming_pool",
        "energy_class",
        "municipality",
    ]
]

# df_selection = df_selection.replace(["apartment", "house"], [0, 1])
# Switching price to log_price


# Dropping missing values in living_area and price
df_selection = df_selection.dropna(subset=["living_area", "price"])
df_selection["log_price"] = df_selection["price"].apply(np.log10)
df_selection = df_selection.drop(columns=["price"])
# Fill missing values in swimming_pool with zeros.
df_selection["swimming_pool"] = df_selection["swimming_pool"].fillna(0)


df_municipality = pd.get_dummies(df_selection[["municipality"]])


X = pd.concat(
    [
        df_selection[
            ["living_area", "terrace", "garden", "swimming_pool", "energy_class"]
        ],
        df_municipality,

    ],
    axis=1,
)
y = df_selection["log_price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

y_pred_full = 10**y_pred
y_test_full = 10**y_test

df_preds = pd.DataFrame(
    {"Actual": y_test_full.squeeze(), "Predicted": y_pred_full.squeeze()}
)
print(df_preds)

         Actual     Predicted
1232   185000.0  2.069097e+05
4421   995000.0  7.879369e+05
36     795000.0  1.177283e+06
4208   469000.0  3.874654e+05
5796  2300000.0  1.974842e+06
...         ...           ...
2356   230000.0  2.365502e+05
6561   230000.0  2.256790e+05
3207   495000.0  2.710124e+05
526    255000.0  2.347042e+05
2243   295000.0  3.629208e+05

[1645 rows x 2 columns]


In [10]:
from sklearn import metrics

print("R²:", metrics.r2_score(y_test, y_pred))

R²: 0.7270621251247988
